In [1]:
from pathlib import Path
from tqdm import tqdm
import requests
import csv

# Better than pandas! Has actual types
import polars as pl

In [2]:
with Path("./landlocked 2023-06-26 - Лист1.csv").open() as file:
    reader = csv.DictReader(file)
    countries = list(map(lambda row: row["landlocked country"], reader))
countries

['Switzerland',
 'Austria',
 'Czech Republic',
 'Hungary',
 'Serbia',
 'Slovakia',
 'Ethiopia',
 'Kazakhstan',
 'Luxembourg',
 'Belarus',
 'Uganda',
 'Nepal',
 'Azerbaijan',
 'Armenia',
 'North Macedonia',
 'Uzbekistan',
 'Zimbabwe',
 'Malawi',
 'Zambia',
 'Moldova',
 'Botswana',
 'Burkina Faso',
 'Rwanda',
 'Mongolia',
 'Bolivia',
 'Paraguay',
 'Mali',
 'Kyrgyzstan',
 'Laos',
 'Afghanistan',
 'Niger',
 'Liechtenstein',
 'Tajikistan',
 'Eswatini',
 'Bhutan',
 'Burundi',
 'Lesotho',
 'Chad',
 'Central African Republic',
 'Vatican City',
 'South Sudan',
 'San Marino',
 'Andorra',
 'Turkmenistan']

In [5]:
def wbsearchentities(name: str):
    response = requests.get(
        "https://www.wikidata.org/w/api.php",
        params={
            "action": "wbsearchentities",
            "search": name,
            "language": "en",
            "format": "json",
        },
    )
    response.raise_for_status()
    return response.json()